# Example 4: Pulse Parametrization

$
\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2}\mid{#2}\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}
$

This example illustrates the parametrization of control pulses as a
form of amplitude constraint.

In [ ]:
using DrWatson
@quickactivate "KrotovTests"

In [ ]:
using QuantumControl
using QuantumControl.Shapes: flattop
using Krotov: SquareParametrization, TanhParametrization, TanhSqParametrization, LogisticParametrization, LogisticSqParametrization
using LinearAlgebra


using PyPlot
matplotlib.use("Agg")

## Parametrizations

## Symmetric Bounded Controls

In [ ]:
function plot_symmetric_parametrization_comparison()
    fig, axs = matplotlib.pyplot.subplots(figsize=(16, 3), ncols=3)

    u_vals = collect(range(-3, 3, length=101))
    ϵ_vals = collect(range(-1, 1, length=101))

    ϵ_min = -1.0
    ϵ_max = 1.0

    axs[1].plot(u_vals, u_vals, "--", color="black")
    axs[1].plot(u_vals, TanhParametrization(ϵ_min, ϵ_max).epsilon_of_u.(u_vals), label="Tanh")
    axs[1].plot(u_vals, LogisticParametrization(ϵ_min, ϵ_max).epsilon_of_u.(u_vals), label="Logistic(k=1)")
    axs[1].plot(u_vals, LogisticParametrization(ϵ_min, ϵ_max, k=4).epsilon_of_u.(u_vals), label="Logistic(k=4)")
    axs[1].set_ylim(-1.2, 1.2)
    axs[1].set_xlabel("u")
    axs[1].set_ylabel("ϵ")

    axs[2].plot(ϵ_vals, ϵ_vals, "--", color="black")
    axs[2].plot(ϵ_vals, TanhParametrization(ϵ_min, ϵ_max).u_of_epsilon.(ϵ_vals), label="Tanh")
    axs[2].plot(ϵ_vals, LogisticParametrization(ϵ_min, ϵ_max).u_of_epsilon.(ϵ_vals), label="Logistic(k=1)")
    axs[2].plot(ϵ_vals, LogisticParametrization(ϵ_min, ϵ_max, k=4).u_of_epsilon.(ϵ_vals), label="Logistic(k=4)")
    axs[2].set_ylim(-3, 3)
    axs[2].set_ylabel("u")
    axs[2].set_xlabel("ϵ")
    axs[2].legend()

    axs[3].plot(u_vals, [1.0 for _ in u_vals], "--", color="black")
    axs[3].plot(u_vals, TanhParametrization(ϵ_min, ϵ_max).de_du_derivative.(u_vals), label="Tanh")
    axs[3].plot(u_vals, LogisticParametrization(ϵ_min, ϵ_max).de_du_derivative.(u_vals), label="Logistic(k=1)")
    axs[3].plot(u_vals, LogisticParametrization(ϵ_min, ϵ_max, k=4).de_du_derivative.(u_vals), label="Logistic(k=4)")
    axs[3].set_ylim(0, 2)
    axs[3].set_xlabel("u")
    axs[3].set_ylabel("∂ϵ/∂u")

    return fig
end

plot_symmetric_parametrization_comparison()

## Positive (Bounded) Controls

In [ ]:
function plot_positive_parametrization_comparison()
    fig, axs = matplotlib.pyplot.subplots(figsize=(16, 3), ncols=3)

    u_vals = collect(range(-3, 3, length=101))
    ϵ_vals = collect(range(0, 1, length=101))
    ϵ_max = 1.0

    axs[1].plot(u_vals, abs.(u_vals), "--", color="black")
    axs[1].plot(u_vals, TanhSqParametrization(ϵ_max).epsilon_of_u.(u_vals), label="TanhSq")
    axs[1].plot(u_vals, LogisticSqParametrization(ϵ_max).epsilon_of_u.(u_vals), label="LogisticSq(k=1)")
    axs[1].plot(u_vals, LogisticSqParametrization(ϵ_max, k=4.0).epsilon_of_u.(u_vals), label="LogisticSq(k=4)")
    axs[1].plot(u_vals, SquareParametrization().epsilon_of_u.(u_vals), label="Square")
    axs[1].set_ylim(0, 1.2)
    axs[1].set_xlabel("u")
    axs[1].set_ylabel("ϵ")

    axs[2].plot(ϵ_vals, ϵ_vals, "--", color="black")
    axs[2].plot(ϵ_vals, TanhSqParametrization(ϵ_max).u_of_epsilon.(ϵ_vals), label="TanhSq")
    axs[2].plot(ϵ_vals, LogisticSqParametrization(ϵ_max).u_of_epsilon.(ϵ_vals), label="LogisticSq(k=1)")
    axs[2].plot(ϵ_vals, LogisticSqParametrization(ϵ_max, k=4.0).u_of_epsilon.(ϵ_vals), label="LogisticSq(k=4)")
    axs[2].plot(ϵ_vals, SquareParametrization().u_of_epsilon.(ϵ_vals), label="Square")
    axs[2].set_ylim(0, 3)
    axs[2].set_ylabel("u")
    axs[2].set_xlabel("ϵ")
    axs[2].legend()

    axs[3].plot(u_vals, sign.(u_vals), "--", color="black")
    axs[3].plot(u_vals, TanhSqParametrization(ϵ_max).de_du_derivative.(u_vals), label="TanhSq")
    axs[3].plot(u_vals, LogisticSqParametrization(ϵ_max).de_du_derivative.(u_vals), label="LogisticSq(k=1)")
    axs[3].plot(u_vals, LogisticSqParametrization(ϵ_max, k=4.0).de_du_derivative.(u_vals), label="LogisticSq(k=4)")
    axs[3].plot(u_vals, SquareParametrization().de_du_derivative.(u_vals), label="Square")
    axs[3].set_ylim(-2, 2)
    axs[3].set_xlabel("u")
    axs[3].set_ylabel("∂ϵ/∂u")

    return fig
end

plot_positive_parametrization_comparison()

## Two-level Hamiltonian

We consider the Hamiltonian $\op{H}_{0} = - \frac{\omega}{2} \op{\sigma}_{z}$, representing
a simple qubit with energy level splitting $\omega$ in the basis
$\{\ket{0},\ket{1}\}$. The control field $\epsilon(t)$ is assumed to couple via
the Hamiltonian $\op{H}_{1}(t) = \epsilon(t) \op{\sigma}_{x}$ to the qubit,
i.e., the control field effectively drives transitions between both qubit
states.

We we will use

In [ ]:
ϵ(t) = 0.2 * flattop(t, T=5, t_rise=0.3, func=:blackman);

In [ ]:
"""Two-level-system Hamiltonian."""
function hamiltonian(Ω=1.0, ϵ=ϵ)
    σ̂_z = ComplexF64[1 0; 0 -1];
    σ̂_x = ComplexF64[0 1; 1  0];
    Ĥ₀ = -0.5 * Ω * σ̂_z
    Ĥ₁ = σ̂_x
    return (Ĥ₀, (Ĥ₁, ϵ))
end
;

In [ ]:
H = hamiltonian();

The control field here switches on from zero at $t=0$ to it's maximum amplitude
0.2 within the time period 0.3 (the switch-on shape is half a [Blackman pulse](https://en.wikipedia.org/wiki/Window_function#Blackman_window)).
It switches off again in the time period 0.3 before the
final time $T=5$). We use a time grid with 500 time steps between 0 and $T$:

In [ ]:
tlist = collect(range(0, 5, length=500));

In [ ]:
function plot_control(pulse::Vector, tlist)
    fig, ax = matplotlib.pyplot.subplots(figsize=(6, 3))
    ax.plot(tlist, pulse)
    ax.set_xlabel("time")
    ax.set_ylabel("amplitude")
    return fig
end

plot_control(ϵ::T, tlist) where T<:Function =
    plot_control([ϵ(t) for t in tlist], tlist)

plot_control(H[2][2], tlist)

## Optimization target

The `krotov` package requires the goal of the optimization to be described by a
list of `Objective` instances. In this example, there is only a single
objective: the state-to-state transfer from initial state $\ket{\Psi_{\init}} =
\ket{0}$ to the target state $\ket{\Psi_{\tgt}} = \ket{1}$, under the dynamics
of the Hamiltonian $\op{H}(t)$:

In [ ]:
function ket(label)
    result = Dict(
        "0" => Vector{ComplexF64}([1, 0]),
        "1" => Vector{ComplexF64}([0, 1]),
    )
    return result[string(label)]
end
;

In [ ]:
objectives = [
    Objective(initial_state=ket(0), generator=H, target_state=ket(1))
]

## Square-parametrization for positive pulses

In [ ]:
problem = ControlProblem(
    objectives=objectives,
    pulse_options=IdDict(
        ϵ  => Dict(
            :lambda_a => 5,
            :update_shape => t -> flattop(t, T=5, t_rise=0.3, func=:blackman),
            :parametrization => SquareParametrization(),
        )
    ),
    tlist=tlist,
    iter_stop=50,
    chi=QuantumControl.Functionals.chi_ss!,
    J_T=QuantumControl.Functionals.J_T_ss,
    check_convergence= res -> begin (
            (res.J_T < 1e-3)
            && (res.converged = true)
            && (res.message="J_T < 10⁻³")
        ) end
);

In [ ]:
opt_result_positive = optimize(problem, method=:krotov);

In [ ]:
opt_result_positive

We can plot the optimized field:

In [ ]:
plot_control(opt_result_positive.optimized_controls[1], tlist)

## Tanh-Square-Parametrization for positive amplitude-constrained pulses

In [ ]:
problem_tanhsq = ControlProblem(
    objectives=objectives,
    pulse_options=IdDict(
        ϵ  => Dict(
            :lambda_a => 10,
            :update_shape => t -> flattop(t, T=5, t_rise=0.3, func=:blackman),
            :parametrization => TanhSqParametrization(3),
        )
    ),
    tlist=tlist,
    iter_stop=50,
    chi=QuantumControl.Functionals.chi_ss!,
    J_T=QuantumControl.Functionals.J_T_ss,
    check_convergence= res -> begin (
            (res.J_T < 1e-3)
            && (res.converged = true)
            && (res.message="J_T < 10⁻³")
        ) end
);

In [ ]:
opt_result_tanhsq = optimize(problem_tanhsq, method=:krotov);

In [ ]:
opt_result_tanhsq

We can plot the optimized field:

In [ ]:
plot_control(opt_result_tanhsq.optimized_controls[1], tlist)

## Logistic-Square-Parametrization for positive amplitude-constrained pulses

In [ ]:
problem_logisticsq = ControlProblem(
    objectives=objectives,
    pulse_options=IdDict(
        ϵ  => Dict(
            :lambda_a => 1,
            :update_shape => t -> flattop(t, T=5, t_rise=0.3, func=:blackman),
            :parametrization => LogisticSqParametrization(3, k=1.0),
        )
    ),
    tlist=tlist,
    iter_stop=50,
    chi=QuantumControl.Functionals.chi_ss!,
    J_T=QuantumControl.Functionals.J_T_ss,
    check_convergence= res -> begin (
            (res.J_T < 1e-3)
            && (res.converged = true)
            && (res.message="J_T < 10⁻³")
        ) end
);

In [ ]:
opt_result_logisticsq = optimize(problem_logisticsq, method=:krotov);

We can plot the optimized field:

In [ ]:
plot_control(opt_result_logisticsq.optimized_controls[1], tlist)

## Tanh-parametrization for amplitude-constrained pulses

In [ ]:
problem_tanh = ControlProblem(
    objectives=objectives,
    pulse_options=IdDict(
        ϵ  => Dict(
            :lambda_a => 1,
            :update_shape => t -> flattop(t, T=5, t_rise=0.3, func=:blackman),
            :parametrization => TanhParametrization(-0.5, 0.5),
        )
    ),
    tlist=tlist,
    iter_stop=50,
    chi=QuantumControl.Functionals.chi_ss!,
    J_T=QuantumControl.Functionals.J_T_ss,
    check_convergence= res -> begin (
            (res.J_T < 1e-3)
            && (res.converged = true)
            && (res.message="J_T < 10⁻³")
        ) end
);

In [ ]:
opt_result_tanh = optimize(problem_tanh, method=:krotov);

In [ ]:
plot_control(opt_result_tanh.optimized_controls[1], tlist)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*